In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def fetch_wind_data(lat, lon, hours=48):
    """
    Fetch hourly wind speed and direction from Open-Meteo API.
    Returns a pandas DataFrame with columns: time, wind_speed, wind_direction
    """
    # Time range: now to now + hours
    start_time = datetime.utcnow()
    end_time = start_time + timedelta(hours=hours)

    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "wind_speed_10m,wind_direction_10m",
        "start_date": start_time.strftime("%Y-%m-%d"),
        "end_date": end_time.strftime("%Y-%m-%d"),
        "timezone": "UTC"
    }

    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame({
        "time": data["hourly"]["time"],
        "wind_speed": data["hourly"]["wind_speed_10m"],
        "wind_direction": data["hourly"]["wind_direction_10m"]
    })

    df["time"] = pd.to_datetime(df["time"])

    return df

In [3]:
def get_wind_conditions_from_df(df_wind, step, interval_minutes=60):
    """
    Retrieve wind speed and direction for the current step based on the DataFrame.
    Assumes 1 step = interval_minutes (e.g., 60 mins).
    """
    idx = min(step, len(df_wind) - 1)
    row = df_wind.iloc[idx]
    return row["wind_speed"], row["wind_direction"]

In [4]:
def interpolate_wind(df_hourly, freq='5min'):
    """
    Interpolate wind speed and direction from hourly to 5-minute intervals.
    """
    df = df_hourly.copy()
    df = df.set_index('time')
    df_resampled = df.resample(freq).interpolate(method='linear')
    df_resampled = df_resampled.reset_index()
    return df_resampled

In [5]:
# Assume the wind is uniformly across all grids
LAT_CENTER, LON_CENTER = 34.072382, -118.601048  # Simulation center

# Fetch hourly wind conditions
df_hourly = fetch_wind_data(LAT_CENTER, LON_CENTER, hours=24)

# Interpolate to 5 minutes interval
df_5min = interpolate_wind(df_hourly, freq='5min')

/var/folders/j6/sbqqj8q50l13h4xp4_6db0v80000gn/T/ipykernel_54475/754439564.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow()


In [6]:
# To get wind speed and direction at each step of simulation
wind_speed, wind_direction = get_wind_conditions_from_df(df_5min, 5) # step 5

print("Wind speed:", wind_speed, "m/s")
print("Wind direction:", wind_direction, "degree")

Wind speed: 9.583333333333334 m/s
Wind direction: 227.33333333333334 degree
